# Get familiar with the Scattering transforms（散射网络初识）
在这个实验中，我们将学习什么是小波散射变换，并对其有一个初步的认识

*在这个实验中我们将使用 [St´ephane Mallat](https://www.di.ens.fr/~mallat/mallat.html) 等人开发的散射网络标准库 [kymatio](https://www.kymat.io/) 来帮助我们更好的学习散射变换并将在后面的实验中帮助我们搭建更多的模型*

Note here, this notebook is just a warm-up exrecise. You will:

- get familiar with the **scattering transform**
- learn how to use the library '**kymatio**' to add the scattring transform to your own model
- **visualize** the different setting for scattring parameter **J**

# What's this Scattering Transform?


首先非常自然的一个问题就是——什么是散射变换，Um... 这个问题好像有点难回答，让我们先参考一下官方的回答：

[A scattering transform is a non-linear signal representation that builds invariance to geometric transformations while preserving a high degree of discriminability. These transforms can be made invariant to translations, rotations (for 2D or 3D signals), frequency shifting (for 1D signals), or changes of scale. These transformations are often irrelevant to many classification and regression tasks, so representing signals using their scattering transform reduces unnecessary variability while capturing structure needed for a given task. This reduced variability simplifies the building of models, especially given small training sets.](https://www.kymat.io/userguide.html)

简单翻译一下：散射变换是一种非线性信号表示，它在保持对几何变换的不变性的情况下，同时保持高度的可辨别性。这种变换可以对平移、旋转（对于 2D 或 3D 信号）、频移（对于 1D 信号）或比例变化保持不变。这种变换通常与许多分类和回归任务无关，因此使用散射变换表示信号可以减少不必要的可变性，同时捕获给定任务所需的结构。这种减少的可变性简化了模型的构建，尤其是在训练集较小的情况下。

好像还是有点难以理解？

或许这样子讲，所谓散射变换（或者说小波散射网络）就可以理解为一种“算子”，这个算子作为一种非线性的信号表示,具有一些比较好的性质，比如上面说的平移不变性，能量保持性，非拓展性等。如果你散射变换的具体描述有兴趣的话，可以参考[AndenM14](https://arxiv.org/pdf/1304.6763.pdf).

我们从这个东西的构成的这个角度出发，这个算子就是由一个基于复数小波变换的卷积神经网络所定义的，然后这个网络又是由二维复数小波变换（比如Gabor变换）和复数取模两种算子交替作用构成的，如下展示了散射网络的前两层结构:

![avatar](./images/scat_transform.png)

如上图，我们可以看到这个网络的每一层其实是先做一个小波变换 $x*\psi_\lambda$ (这里 $\psi_\lambda$ 表示小波算子，也就是输入信号与小波算子做卷积)，然后结果取模，即 $\left | x*\psi_\lambda  \right | $, 最后这一层的输出再通过一个高斯低通滤波器 $\phi_J$, 故而该层的输出是 $\left | x*\psi_\lambda  \right |*\phi_J $

最后由于这个网络由多层组成（一般是3层），我们级联每一层的输出作为散射网络的最终输出~~~

okay, 到这里我们理论描述部分结束了，如果你想获得对散射变换的更一般的定义(高阶的数学描述QAQ~)，可以参考[Mal12](https://arxiv.org/abs/1101.2286).

# How should I use the scattering transform

可恶的理论部分已经过去了，现在让我们开始快乐的写代码吧！

大佬们已经做出了一个散射网络得标准库 [kymatio](https://www.kymat.io/) ，所以让我们来一探究竟~~~

首先，我们必须安装这包，考虑到这个库的API支持非常多的主流的库的前端范式，这里的前端的意思就是我们可以基于不用的库来使用这个包，比如numpy或者sklearn，又或者深度学习中的比较流行的库和pytorch和tensorflow (*PS: 这就是大佬们的力量吧 *)。

同时，考虑到pytorch比较简单, 我们接下来的实验都基于pytorch来写。

推荐大家使用conda来建立一个虚拟环境, 请大家务必先在虚拟环境里面先安装好pytorch等包, 然后从这个虚拟环境里重新启动notebook。

如果不想麻烦，或许直接复制我的这个虚拟环境也是可以的，如下，我们需要先执行下面这个语句来安装

当然你也可以自己配置虚拟环境，毕竟本实验不需要安装太多的库，大概就是**pytorch，matplotlib，numpy, PIL, scipy** 以及我们的需要用的散射网络库kymatio.(以上列出来的库可能不太全面，但我想应该问题不大了~）

如果你复制我的虚拟环境遇到了困难，或许可以直接手动安装这些库，速度可能会更快，毕竟我的环境的里面还有一些其他的库。

anyway, 这些都是细琐的环境问题~

In [ ]:
# The first step, make sure you have installed the required lib，Note it is Optional !!!
# ! pip install -r requirement.txt

## How to use this library?

请先确保你已经成功安装了pytorch！

在这里我们使用torch这个前端接口，这个前端接口其实就是使用torch.nn.Module实现的，这个库的厉害之处就在你如果使用他的这个torch的接口的话，你就可以已将你的散射网络的模块集合到其他的pytorch的模型之中， 同时它还支持cuda() 这个方法，也就是说我们可以用GPU加速！

下面我们基于pytorch演示一下：

In [ ]:
# 下面这行命令可能不是必须的！！!
# 如果你的 Notebook 老是挂掉然后重启的话可能这行命令是有用的~
import os 
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [ ]:
# the first step, load the lib
from kymatio.torch import Scattering2D

# Initialize the scatting transform object
scattering = Scattering2D(J=2,shape=(32,32))

In [ ]:
# try a liitle example...
import torch

# 先建立一张灰度图像
x = torch.randn(1,1,32,32)

# 对其进行散射变换
sx = scattering(x)

print(f"The original input(with shape :{x.shape}):\n",x)
print(f"The data after scatting transform (with shape: {sx.shape}):\n", sx)

# Be careful about the changes of the data type

在使用散射变换时，我们需要注意数据经过散射变换十之后其实是数据量是变大了的

具体来说，假设我们输入是一张二维的灰度图像，它的大小是（B,C,H,W）,B代表batch-size,C代表通道数，RGB图像为3，灰度图像为1，HxW代表图像的分辨率了，这里不再赘述~，那么散射网络的输出大小应当是 

$$ (B,C,\frac{1+LJ+L^{2}J(J-1)}{2},\frac{H}{2^J},\frac{W}{2^J} )$$

这里，L表示散射网络种旋转角度的个数，默认为8，然后J代表散射网络中的空间窗口的缩放系数，默认为2。

如上面的例子，我们输入的大小是（1，1，32，32），那么其散射变换之后提取的特征大小应当是（1, 1, $1+2*8*64$, 32/4, 32/4）

## Comparation of different parameter settings for J
在使用kymatio库的时候，除了选择合适的前端（我们上面选的就是numpy），我们一般只需要根据实际情况选择不同的参数J（一般而言，这个参数越小表明散射变换之后的得到的特征越精细)。

下面我们训练一个小的模型并在这个过程中对比一下不同参数J的效果

In [ ]:
# 首先，加载一些库
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn.functional as F
from PIL import Image
from torch import optim
from scipy.misc import face

from kymatio.torch import Scattering2D

device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# 加载我们实验所需要的测试图片
src_img = Image.fromarray(face())
src_img = src_img.resize((512, 384), Image.ANTIALIAS)
src_img = np.array(src_img).astype(np.float32)
src_img = src_img / 255.0
plt.imshow(src_img)
plt.title("Original image")

src_img = np.moveaxis(src_img, -1, 0)  # image size for HWC to CHW
max_iter = 5 # number of steps for the GD
print("Image shape: ", src_img.shape)
channels, height, width = src_img.shape

## Your exercise!!!

注意在下面的代码是**不完整的**!!!

你只需要运用我们之前学习的例子就可以完成它！


In [ ]:

for J in [2, 4, 6]:

    src_img_tensor = torch.from_numpy(src_img).to(device).contiguous()

    ##############################################################################
    # TODO: Define a Sccattering object and then use it to compute the                   #
    # scattering coefficient with result to the 2D images we load before                   #
    # Note here： 这里我们需要你计算我们之前加载的测试图片的散射系数，并在之后会将其   #
    #                                作为目标成为损失函数的一部分去训练一个小的模型 ~                   #
    ##############################################################################
    # store the scattering coefficient in this object
    scattering_coefficients = None
    
    ##############################################################################
    #                             END OF YOUR CODE                               #
    ##############################################################################

    max_iter = 20
    # Create trainable input image
    input_tensor = torch.rand(src_img.shape, requires_grad=True, device=device)
    # Optimizer hyperparams
    optimizer = optim.Adam([input_tensor], lr=1)
    # The loss function
    loss_func = torch.nn.MSELoss()
    
    # Training
    best_img = None
    best_loss = float("inf")
    for epoch in range(1, max_iter):
        ##############################################################################
        # TODO: 我们希望你能够运用我们之前计算的散射系数作为目标去训练我们随机生成的输入张量    #
        #               使得输入张量的散射系数和测试图片的散射系数的 mes_loss 尽可能地小，这和一般  #
        #               的深度学习过程有点好像有些不太一样？ 但是我想基本思路应该是类似的         #
        ##############################################################################
        
        ##############################################################################
        #                             END OF YOUR CODE                               #
        ##############################################################################
        print("Epoch {}, loss: {}".format(epoch, loss.item()), end="\r")
        if loss < best_loss:
            best_loss = loss.detach().cpu().item()
            best_img = input_tensor.detach().cpu().numpy()

    best_img = np.clip(best_img, 0.0, 1.0)

    # PSNR
    mse = np.mean((src_img - best_img) ** 2)
    psnr = 20 * np.log10(1.0 / np.sqrt(mse))
    print("\nPSNR: {:.2f}dB for J={}".format(psnr, J))

    # Plot
    plt.figure()
    plt.imshow(np.moveaxis(best_img, 0, -1))
    plt.title("PSNR: {:.2f}dB (J={})".format(psnr, J))

plt.show()

# How about your result?

至少直观的来看，我我们发现 $J$ 越大那么经过散射提取的特征得到的图片看起来也越不精细（空间缩放尺度越大自然得到的散射特征也越不细致），这和理论是相符的~

我的结果是这个样子的， 或许可以给你一个参考:
![avatar](./images/assignment_1_1.png)
![avatar](./images/assignment_1_2.png)
![avatar](./images/assignment_1_3.png)

啊哈，到这里散射网络部分的第一部分就结束了，下一个实验将在 assignment_2, 祝你好运！
